<a href="https://colab.research.google.com/github/moon-123/NLP-Okt/blob/main/%08%ED%95%9C%EA%B5%AD%EC%96%B4_NLP_%EA%B3%BC%EC%A0%9C(Okt%2C_%EB%AA%85%EC%82%AC%2B%EB%8F%99%EC%82%AC%EC%9B%90%ED%98%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 문화, 게임 콘텐츠 분야 용어 말뭉치

# 2. 데이터 불러오기

In [4]:
import json

with open('/content/drive/MyDrive/KDT/자연어 처리/data/용례_레저.json', 'r') as file:
    json_data = json.load(file)


In [5]:
json_data[0]

{'id': 3087876,
 'sentence': '김치볶음 해놓은거에 밥비벼먹고 카레 해먹고',
 'tokens': [{'start': 0,
   'length': 4,
   'sub': '김치볶음',
   'facet': '구체물',
   'term_id': 134217,
   'sense_no': 1}],
 'sense_no': 1,
 'source': {'uri': 'https://bbs.ruliweb.com/community/board/300143/read/58933395',
  'text': '일주일동안 탄수화물 위주로만 먹었어\r\n\r\n\r\n김치볶음 해놓은거에 밥비벼먹고 카레 해먹고\r\n\r\n\r\n점심에는 김밥에 쫄면에 돈까스 먹었는데\r\n\r\n\r\n그래도 단백질이 먹고싶다',
  'written_at': '2022-10-15T08:04:00'}}

### 2-1. 'sentence'
* 'sentence'는 source text에서 추출된 문장.

In [6]:
# source의 text를 정제/추출해도 되는데 과제는 전처리가 아니라 임베딩이므로 생략
# sentence를 사용
sentence_list = [data['sentence'] for data in json_data]
print(sentence_list[:5])
print(len(sentence_list))

['김치볶음 해놓은거에 밥비벼먹고 카레 해먹고', '전과하고 싶어지네 전적대에서 다니던 과로 전과할까 해도 이번학기 학점이 1~2점 나올거 같아서 재편입 해야하나 싶다...', '중요한건 저기 예시로 든건 노트북용도 아닌 컴터용 시퓨다', '의자를 좀 더 편한걸로 바꿔주면 딱이네 빈백 같은거', '수석이니하면서 장미빛 미래를 꿈꾸고 있을']
75384


### 2-2. 한국어 토크나이저 Okt사용


In [7]:
!pip install KoNLPy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 27.4 MB/s eta 0:00:00


In [8]:
from konlpy.tag import Okt
okt = Okt()

In [27]:
# tokenized_sentence = okt.pos(sentence_list[0], norm=True, stem=True)
# tokenized_sentence[0][1] == 'Noun'

def extract_nouns_and_verbs(text):
    pos_sentence = []
    pos_tags = okt.pos(text, norm=True, stem=True)

    pos_sentence = [word for word, pos in pos_tags if (pos == 'Noun') or (pos == 'Verb')]


    return pos_sentence

In [28]:
tokenized_sentence_list = []

for sentence in sentence_list:
    tokenized_sentence_list.append(extract_nouns_and_verbs(sentence))

# tokenized_sentence_list
# 명사, 동사원형을 제외하고 삭제

### 2-2. 불용어, 특수문자 제거
* okt에서 명사만 뽑았기 때문에 특수문자 제거는 생략

In [30]:
stopwords_list = set(open('/content/drive/MyDrive/KDT/자연어 처리/data/stopword.txt').read().split('\n')).union(['거', '점', '걸', '더', '게', '것', '데', '참', '뭘', '쭉'])
len(stopwords_list)
pre_sentence_list = []
for sentence in tokenized_sentence_list:
    word_list = []
    for word in sentence:
        if word not in stopwords_list:
            word_list.append(word)

    pre_sentence_list.append(word_list)

len(pre_sentence_list)

75384

In [31]:
# 제거 후 확인
print(pre_sentence_list[:5])

[['김치', '볶음', '밥', '비비다', '먹다', '카레', '해먹'], ['전과', '싶다', '전적', '다니다', '과로', '전과', '해도', '학기', '학점', '나오다', '재', '편입', '싶다'], ['예시', '들다', '노트북', '용도', '컴터', '용', '시퓨'], ['의자', '바꾸다', '빈'], ['수석', '장미', '빛', '미래', '꿈꾸다']]


In [34]:
# 빈 리스트 제거
sentence_list = []
for ps in pre_sentence_list:
    if len(ps) != 0:
        sentence_list.append(ps)

print(len(sentence_list))

75381


In [35]:
print(sentence_list[:5])

[['김치', '볶음', '밥', '비비다', '먹다', '카레', '해먹'], ['전과', '싶다', '전적', '다니다', '과로', '전과', '해도', '학기', '학점', '나오다', '재', '편입', '싶다'], ['예시', '들다', '노트북', '용도', '컴터', '용', '시퓨'], ['의자', '바꾸다', '빈'], ['수석', '장미', '빛', '미래', '꿈꾸다']]


### 2-3. Tokenizer로 단어에 라벨링

In [36]:
# pre_sentence_list를 가지고 라벨링
# 지금 리스트니까 str 문장으로 변환해야할까?

from tensorflow.keras.preprocessing.text import Tokenizer

In [37]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence_list)

In [38]:
word2idx = tokenizer.word_index
# print(word2idx)
print(len(word2idx))

35173


In [39]:
idx2word = {value : key for key, value in word2idx.items()}

vocab_size = len(idx2word)
print(vocab_size)

35173


In [40]:
encoded = tokenizer.texts_to_sequences(sentence_list)
print(encoded[:2])

[[76, 187, 37, 1005, 1, 613, 817], [4353, 24, 4690, 94, 11342, 4353, 172, 3770, 8710, 3, 60, 3099, 24]]


In [41]:
len(encoded)

75381

### 2-4. skipgrams로 임베딩 준비

In [42]:
from tensorflow.keras.preprocessing.sequence import skipgrams

In [43]:
# encoded에서 랜덤하게 리스트를 슬라이싱하기
import random
random_encoded_100 = random.sample(encoded, 100)


In [44]:
# edcoded 총 개수가 33997.
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=5) for sample in random_encoded_100]
print(f'{len(skip_grams)}')

100


In [47]:
for i, sg in enumerate(skip_grams):
    if len(sg[0]) <= 2:
        print(f'{i}: {sg}')

# 빈 값이 있는 것 같아서 확인함
# 명사만 추출한 경우엔 윈도우 사이즈가 적당해야했지만 명사와 동사원형을 추출하다보니 윈도우 사이즈에 제약이 줄었음.

### 2-5. 학습을 위한 모델 만들기

In [48]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dot
from tensorflow.keras.utils import plot_model

In [49]:
embedding_dim = 100

w_inputs = Input(shape=(1,), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

c_inputs = Input(shape=(1,), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_dim)(c_inputs)


In [50]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 100)               3517300   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 100)               3517300   ['input_2[0][0]']             
                                                                                              

In [51]:
model.compile(loss='binary_crossentropy', optimizer='adam')

### 2-6. 학습

In [52]:
import numpy as np

In [53]:
for epoch in range(100):
    loss = 0
    for _, elem in enumerate(skip_grams):
        if elem[0]:
            first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
            second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
            labels = np.array(elem[1], dtype='int32')
            X = [first_elem, second_elem]
            Y = labels
            loss += model.train_on_batch(X, Y)
    print('Epoch: ', epoch+1, 'Loss: ', loss)


Epoch:  1 Loss:  69.31307345628738
Epoch:  2 Loss:  68.40207976102829
Epoch:  3 Loss:  66.90074932575226
Epoch:  4 Loss:  64.30526381731033
Epoch:  5 Loss:  60.352422058582306
Epoch:  6 Loss:  55.126506835222244
Epoch:  7 Loss:  48.99506130814552
Epoch:  8 Loss:  42.45842531323433
Epoch:  9 Loss:  36.00636710226536
Epoch:  10 Loss:  30.025200575590134
Epoch:  11 Loss:  24.75468309223652
Epoch:  12 Loss:  20.28902331739664
Epoch:  13 Loss:  16.610417380928993
Epoch:  14 Loss:  13.634554840624332
Epoch:  15 Loss:  11.25035334378481
Epoch:  16 Loss:  9.346167247742414
Epoch:  17 Loss:  7.82304173707962
Epoch:  18 Loss:  6.599111013114452
Epoch:  19 Loss:  5.6092375963926315
Epoch:  20 Loss:  4.802680717781186
Epoch:  21 Loss:  4.140311267226934
Epoch:  22 Loss:  3.5920459926128387
Epoch:  23 Loss:  3.1347236577421427
Epoch:  24 Loss:  2.7504370771348476
Epoch:  25 Loss:  2.425260222516954
Epoch:  26 Loss:  2.148289058357477
Epoch:  27 Loss:  1.9109234623610973
Epoch:  28 Loss:  1.70632998

In [54]:
import gensim


In [59]:
f = open('vectors.txt', 'w')
f.write('{} {}\n'.format(vocab_size, embedding_dim))
vectors = model.get_weights()[0]

print(vectors)

for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i-1, :])))))

[[-0.02698928  0.00931174  0.03408981 ...  0.02772317 -0.03054793
   0.00105447]
 [-0.59130096 -0.8258808  -0.6383123  ...  0.70729196  0.17827253
  -0.6512729 ]
 [ 0.30506632 -0.61675763 -0.5821475  ...  0.5824413  -0.5917793
  -0.7724286 ]
 ...
 [-0.01168431  0.01931765 -0.00242277 ... -0.04077073  0.02409116
   0.01764495]
 [-0.02997527  0.00955244  0.01048    ... -0.04364777 -0.01908641
   0.04691038]
 [-0.03582549  0.02726832 -0.01737507 ...  0.02078445 -0.04168485
  -0.02958456]]


In [60]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [70]:
w2v.most_similar(positive=['악어'])

[('뽀록나다', 0.43311867117881775),
 ('오피스', 0.3811410665512085),
 ('순두부찌개', 0.3756980895996094),
 ('비사', 0.37259188294410706),
 ('플래시', 0.36377960443496704),
 ('미텐스', 0.36125078797340393),
 ('은줄', 0.360832154750824),
 ('롤드컵', 0.3543761968612671),
 ('작문', 0.35200151801109314),
 ('등장인물', 0.3454458713531494)]

In [71]:
w2v.most_similar(positive=['눈물'])

[('글카', 0.41196003556251526),
 ('별하늘', 0.38758835196495056),
 ('방물', 0.3807717561721802),
 ('예감', 0.37183961272239685),
 ('어솨', 0.3666045665740967),
 ('직촬보', 0.36536285281181335),
 ('생지옥', 0.3575366735458374),
 ('신앙심', 0.3501545786857605),
 ('눈시울', 0.34760603308677673),
 ('직빵임', 0.34690073132514954)]